In [21]:

import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException

from selenium.webdriver.common.by import By

In [22]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.imdb.com/search/title/?moviemeter=1,1000')

In [23]:
# Hàm lọc
def filter_array(array_elements):
    array = []
    for element in array_elements:
        array_processed = element.text.replace('\n', '').strip()
        array.append(array_processed)
    return array

# Đợi cho phần tử "See more" hiển thị
wait = WebDriverWait(driver, 10)
see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))

# Cuộn trang web xuống tới cuối
while see_more_button.is_displayed():
    actions = ActionChains(driver)
    actions.move_to_element(see_more_button).perform()
    try:
        driver.execute_script("arguments[0].click();", see_more_button)
        see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))
    except:
        break


In [24]:
# Lọc description
description = []
description_element = None
try:
    description_element = driver.find_element(By.CSS_SELECTOR, 'div.ipc-html-content.ipc-html-content--base.sc-74bf520e-0.iiRKKi.dli-plot-container div.ipc-html-content-inner-div')
except NoSuchElementException:
    pass

description_text = description_element.text if description_element else None
description.append(description_text)
# Lọc movie link
movie_link_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-title-link-wrapper')
movie_links = [element.get_attribute('href') for element in movie_link_elements]

movies = []
release_date = []
film_actors = []
duration = []
country = []
companies = []
movie_category = []
ratings = []
directors = []
Certificate = []
budget= []
boxoffice= []
original_language = []
taglines = []
writing_credits = []

In [25]:

for movie_link in movie_links:
    driver.get(movie_link)
    # Lọc movie
    try:
        movie = driver.find_element('css selector', 'h1[data-testid="hero__pageTitle"] span.hero__primary-text')
        if movie:
            movies.append(movie.text)  
        else:
            movies.append(None)  
    except:
        movies.append(None)  
    # Lọc diễn viên
    actor_elements = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="title-cast-item__actor"]')
    actor_text = ', '.join([element.text for element in actor_elements])
    film_actors.append(actor_text)
    # Lọc duration
    duration_element = None
    try:
        duration_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-techspec_runtime"] div.ipc-metadata-list-item__content-container')
    except NoSuchElementException:
        pass
    duration_text = duration_element.text if duration_element else None
    duration.append(duration_text)
    #Lọc ratings
    rating_elements = None
    try:
        rating_elements = driver.find_element(By.CSS_SELECTOR,'span.sc-d541859f-1.imUuxf')
    except NoSuchElementException:
        pass
    rating_text = rating_elements.text if rating_elements else None
    ratings.append(rating_text)
    # Lọc release date
    release_date_element = None
    try:
        release_date_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-releasedate"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    release_date_text = release_date_element.text if release_date_element else None
    release_date.append(release_date_text)
    # Lọc country
    country_element = None
    try:
        country_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-origin"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    country_text = country_element.text if country_element else None
    country.append(country_text)
    # Lọc budget và box office
    budget_element = None
    boxoffice_element = None
    try:
        budget_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-budget"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
        boxoffice_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-cumulativeworldwidegross"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
    except NoSuchElementException:
        pass
    budget_text = budget_element.text if budget_element else None
    budget.append(budget_text)
    boxoffice_text = boxoffice_element.text if boxoffice_element else None
    boxoffice.append(boxoffice_text)
    #Lọc Certificate
    certificate_element = None
    try:
        certificate_element = driver.find_element(By.XPATH, '//a[contains(@href, "/parentalguide/certificates")]')
    except NoSuchElementException:
        pass

    certificate_text = certificate_element.text if certificate_element else None
    Certificate.append(certificate_text)

    # Lọc original_language
    original_language_element = None
    try:
        original_language_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-languages"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    
    original_language_text = original_language_element.text if original_language_element else None
    original_language.append(original_language_text)
    # Lọc companies
    companies_elements = None
    try:
        companies_elements = driver.find_elements(By.CSS_SELECTOR,'li[data-testid="title-details-companies"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li')
    except NoSuchElementException:
        pass

    companies_text = ''
    for i, element in enumerate(companies_elements):
        if i < len(companies_elements) - 1:
            companies_text += element.text + ', '
        else:
            companies_text += element.text

    companies.append(companies_text)
    # Lọc genre
    movie_category_elements = None
    movie_category_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-chip.ipc-chip--on-baseAlt span.ipc-chip__text')
    movie_category_text = ', '.join([element.text for element in movie_category_elements]) if movie_category_elements else None
    movie_category.append(movie_category_text)
    #print(movie_category)
    #Lọc director
    try:
        full_cast_crew_link = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="See full cast and crew"]')
        driver.execute_script("arguments[0].click();", full_cast_crew_link)
        time.sleep(3)  # wait for the page to load
        
        director_tag_element = driver.find_element(By.CSS_SELECTOR, 'h4[id="director"]')
        table_element = director_tag_element.find_element(By.XPATH, './following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        director_elements = table_element.find_elements(By.CSS_SELECTOR, 'td.name a[href^="/name/"]')
        directors_text = ', '.join([element.text for element in director_elements]) if director_elements else None
        directors.append(directors_text)
    except NoSuchElementException:
        directors.append(None)
    # Lọc Writing Credits
    writing_credits_tag_element = None
    try:
        writing_credits_tag_element = driver.find_element(By.CSS_SELECTOR,'h4[id="writer"]')
    except NoSuchElementException:
        pass

    if writing_credits_tag_element:
        table_element = writing_credits_tag_element.find_element(By.XPATH,'./following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        writing_credits_element = table_element.find_elements(By.CSS_SELECTOR,'td.name a[href^="/name/"]')
    else:
        writing_credits_element = None

    writing_credits_text = ', '.join([element.text for element in writing_credits_element]) if writing_credits_element else None
    writing_credits.append(writing_credits_text)


In [26]:
# Define the expected length
expected_length = len(movie_links)

# Function to pad lists with None
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to the same length
movies = pad_list(movies, expected_length)
film_actors = pad_list(film_actors, expected_length)
duration = pad_list(duration, expected_length)
ratings = pad_list(ratings, expected_length)
release_date = pad_list(release_date, expected_length)
country = pad_list(country, expected_length)
budget = pad_list(budget, expected_length)
boxoffice = pad_list(boxoffice, expected_length)
certificate = pad_list(Certificate, expected_length)
original_language = pad_list(original_language, expected_length)
companies = pad_list(companies, expected_length)
movie_category = pad_list(movie_category, expected_length)
directors = pad_list(directors, expected_length)
writing_credits = pad_list(writing_credits, expected_length)


In [27]:
import pandas as pd

# Create a dictionary of the lists
data = {
    'Links' : movie_links,
    'Movie': movies,
    'Actors': film_actors,
    'Duration': duration,
    'Ratings': ratings,
    'Release Date': release_date,
    'Country': country,
    'Budget': budget,
    'Box Office': boxoffice,
    'Certificate': certificate,
    'Original Language': original_language,
    'Companies': companies,
    'Genres': movie_category,
    'Directors': directors,
    'Writing Credits': writing_credits
}

# Create the DataFrame
df = pd.DataFrame(data)

# Output the DataFrame to check

# Export the DataFrame to a CSV file
df.to_csv('movies_data.csv', index=False)

print("DataFrame successfully exported to movies_data.csv")


DataFrame successfully exported to movies_data.csv


In [28]:
df

,Links,Movie,Actors,Duration,Ratings,Release Date,Country,Budget,Box Office,Certificate,Original Language,Companies,Genres,Directors,Writing Credits
0,https://www.imdb.com/title/tt15435876/?ref_=sr...,The Penguin,"Colin Farrell, Cristin Milioti, Rhenzy Feliz, ...",None,8.8,"September 19, 2024 (United States)",United States,None,None,TV-MA,English,"6th & Idaho Productions, Chapel Place Producti...","Gangster, Tragedy, Crime, Drama","Craig Zobel, Kevin Bray, Helen Shaver, Jennife...","Bill Finger, Bob Kane, Lauren LeFranc, Lauren ..."
1,https://www.imdb.com/title/tt7737800/?ref_=sr_t_2,Hẹn Hò Với Sát Nhân,"Anna Kendrick, Daniel Zovatto, Tony Hale, Nico...",1 hour 35 minutes,6.6,"October 4, 2024 (Vietnam)",United States,None,None,T16,English,"AGC Studios, BondIt Media Capital, Vertigo Ent...","Period Drama, Serial Killer, True Crime, Crime...",Anna Kendrick,Ian McDonald
2,https://www.imdb.com/title/tt17526714/?ref_=sr...,Thần Dược,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",2 hours 21 minutes,7.6,"November 1, 2024 (Vietnam)",United Kingdom,"$17,500,000 (estimated)","$42,776,354",R,English,"Working Title Films, A Good Story, Blacksmith","Body Horror, Dark Comedy, Monster Horror, Psyc...",Coralie Fargeat,Coralie Fargeat
3,https://www.imdb.com/title/tt16366836/?ref_=sr...,Venom: Kèo Cuối,"Tom Hardy, Chiwetel Ejiofor, Juno Temple, Rhys...",1 hour 50 minutes,6.2,"October 25, 2024 (Vietnam)",United States,"$120,000,000 (estimated)","$187,339,701",T13,English,"Columbia Pictures, Arad Productions, Hutch Par...","Alien Invasion, Kaiju, Road Trip, Superhero, A...",Kelly Marcel,"Kelly Marcel, Tom Hardy, Kelly Marcel"
4,https://www.imdb.com/title/tt21906238/?ref_=sr...,Rivals,"Alex Hassell, Victoria Smurfit, Bella Maclean,...",55 minutes,8.0,"October 18, 2024 (United Kingdom)",United States,None,None,TV-MA,English,"Happy Prince, Walt Disney Studios",Drama,"Elliot Hegarty, Alexandra Brodski, Dee Koppang...","Dominic Treadwell-Collins, Laura Wade, Dare Ai..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,https://www.imdb.com/title/tt21064584/?ref_=sr...,The Iron Claw,"Zac Efron, Jeremy Allen White, Harris Dickinso...",2 hours 12 minutes,,"December 22, 2023 (United States)",United Kingdom,"$15,000,000 (estimated)","$45,204,449",R,English,"A24, Access Entertainment, BBC Film","Docudrama, Tragedy, Biography, Drama, Sport",Sean Durkin,Sean Durkin
995,https://www.imdb.com/title/tt0119237/?ref_=sr_...,Gummo,"Nick Sutton, Jacob Sewell, Lara Tosh, Jacob Re...",1 hour 29 minutes,,"November 24, 1997 (Australia)",United States,"$1,300,000 (estimated)","$116,799",R,English,"Fine Line Features, Independent Pictures (II)","Dark Comedy, Comedy, Drama",Harmony Korine,Harmony Korine
996,https://www.imdb.com/title/tt16300962/?ref_=sr...,Cuando acecha la maldad,"Ezequiel Rodríguez, Demián Salomón, Silvina Sa...",1 hour 39 minutes,,"October 27, 2023 (United States)",Argentina,None,None,Not Rated,Spanish,"Machaco Films, Aramos Cine, Shudder","Supernatural Horror, Horror",Demián Rugna,Demián Rugna
997,https://www.imdb.com/title/tt0110357/?ref_=sr_...,Vua Sư Tử,"Matthew Broderick, Jeremy Irons, James Earl Jo...",1 hour 28 minutes,,"June 24, 1994 (United States)",United States,"$45,000,000 (estimated)","$981,708,285",G,English,"Walt Disney Pictures, Walt Disney Animation St...","Animal Adventure, Coming-of-Age, Hand-Drawn An...","Roger Allers, Rob Minkoff","Irene Mecchi, Jonathan Roberts, Linda Woolvert..."


In [29]:
# Ensure all lists are padded to the same length before creating the DataFrame
description = pad_list(description, expected_length)

# Create the DataFrame as usual
data = {
    'Links': movie_links,
    'Movie': movies,
    'Actors': film_actors,
    'Duration': duration,
    'Ratings': ratings,
    'Release Date': release_date,
    'Country': country,
    'Budget': budget,
    'Box Office': boxoffice,
    'Certificate': Certificate,
    'Original Language': original_language,
    'Companies': companies,
    'Genres': movie_category,
    'Directors': directors,
    'Writing Credits': writing_credits,
    'Description': description  # Now the lengths should match
}

df = pd.DataFrame(data)
df.to_csv('IMDB_Top_Movies.csv', index=False)

print("DataFrame successfully exported to IMDB_Top_Movies.csv")


DataFrame successfully exported to IMDB_Top_Movies.csv


In [30]:
df

,Links,Movie,Actors,Duration,Ratings,Release Date,Country,Budget,Box Office,Certificate,Original Language,Companies,Genres,Directors,Writing Credits,Description
0,https://www.imdb.com/title/tt15435876/?ref_=sr...,The Penguin,"Colin Farrell, Cristin Milioti, Rhenzy Feliz, ...",None,8.8,"September 19, 2024 (United States)",United States,None,None,TV-MA,English,"6th & Idaho Productions, Chapel Place Producti...","Gangster, Tragedy, Crime, Drama","Craig Zobel, Kevin Bray, Helen Shaver, Jennife...","Bill Finger, Bob Kane, Lauren LeFranc, Lauren ...",None
1,https://www.imdb.com/title/tt7737800/?ref_=sr_t_2,Hẹn Hò Với Sát Nhân,"Anna Kendrick, Daniel Zovatto, Tony Hale, Nico...",1 hour 35 minutes,6.6,"October 4, 2024 (Vietnam)",United States,None,None,T16,English,"AGC Studios, BondIt Media Capital, Vertigo Ent...","Period Drama, Serial Killer, True Crime, Crime...",Anna Kendrick,Ian McDonald,None
2,https://www.imdb.com/title/tt17526714/?ref_=sr...,Thần Dược,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",2 hours 21 minutes,7.6,"November 1, 2024 (Vietnam)",United Kingdom,"$17,500,000 (estimated)","$42,776,354",R,English,"Working Title Films, A Good Story, Blacksmith","Body Horror, Dark Comedy, Monster Horror, Psyc...",Coralie Fargeat,Coralie Fargeat,None
3,https://www.imdb.com/title/tt16366836/?ref_=sr...,Venom: Kèo Cuối,"Tom Hardy, Chiwetel Ejiofor, Juno Temple, Rhys...",1 hour 50 minutes,6.2,"October 25, 2024 (Vietnam)",United States,"$120,000,000 (estimated)","$187,339,701",T13,English,"Columbia Pictures, Arad Productions, Hutch Par...","Alien Invasion, Kaiju, Road Trip, Superhero, A...",Kelly Marcel,"Kelly Marcel, Tom Hardy, Kelly Marcel",None
4,https://www.imdb.com/title/tt21906238/?ref_=sr...,Rivals,"Alex Hassell, Victoria Smurfit, Bella Maclean,...",55 minutes,8.0,"October 18, 2024 (United Kingdom)",United States,None,None,TV-MA,English,"Happy Prince, Walt Disney Studios",Drama,"Elliot Hegarty, Alexandra Brodski, Dee Koppang...","Dominic Treadwell-Collins, Laura Wade, Dare Ai...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,https://www.imdb.com/title/tt21064584/?ref_=sr...,The Iron Claw,"Zac Efron, Jeremy Allen White, Harris Dickinso...",2 hours 12 minutes,,"December 22, 2023 (United States)",United Kingdom,"$15,000,000 (estimated)","$45,204,449",R,English,"A24, Access Entertainment, BBC Film","Docudrama, Tragedy, Biography, Drama, Sport",Sean Durkin,Sean Durkin,None
995,https://www.imdb.com/title/tt0119237/?ref_=sr_...,Gummo,"Nick Sutton, Jacob Sewell, Lara Tosh, Jacob Re...",1 hour 29 minutes,,"November 24, 1997 (Australia)",United States,"$1,300,000 (estimated)","$116,799",R,English,"Fine Line Features, Independent Pictures (II)","Dark Comedy, Comedy, Drama",Harmony Korine,Harmony Korine,None
996,https://www.imdb.com/title/tt16300962/?ref_=sr...,Cuando acecha la maldad,"Ezequiel Rodríguez, Demián Salomón, Silvina Sa...",1 hour 39 minutes,,"October 27, 2023 (United States)",Argentina,None,None,Not Rated,Spanish,"Machaco Films, Aramos Cine, Shudder","Supernatural Horror, Horror",Demián Rugna,Demián Rugna,None
997,https://www.imdb.com/title/tt0110357/?ref_=sr_...,Vua Sư Tử,"Matthew Broderick, Jeremy Irons, James Earl Jo...",1 hour 28 minutes,,"June 24, 1994 (United States)",United States,"$45,000,000 (estimated)","$981,708,285",G,English,"Walt Disney Pictures, Walt Disney Animation St...","Animal Adventure, Coming-of-Age, Hand-Drawn An...","Roger Allers, Rob Minkoff","Irene Mecchi, Jonathan Roberts, Linda Woolvert...",None


In [31]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('IMDB_Top_Movies.csv')
df2 = pd.read_csv('movies_data.csv')

# Concatenate the DataFrames
# If you want to concatenate them vertically (rows added)
merged_df = pd.concat([df1, df2], ignore_index=True)

# If you want to concatenate them horizontally (columns added)
# merged_df = pd.concat([df1, df2], axis=1)

# Save the concatenated DataFrame to a new CSV file
merged_df.to_csv('1000 IMDB.csv', index=False)

print("CSV files successfully merged into merged_file.csv")


CSV files successfully merged into merged_file.csv


In [32]:
merged_df

,Links,Movie,Actors,Duration,Ratings,Release Date,Country,Budget,Box Office,Certificate,Original Language,Companies,Genres,Directors,Writing Credits,Description
0,https://www.imdb.com/title/tt15435876/?ref_=sr...,The Penguin,"Colin Farrell, Cristin Milioti, Rhenzy Feliz, ...",NaN,8.8,"September 19, 2024 (United States)",United States,NaN,NaN,TV-MA,English,"6th & Idaho Productions, Chapel Place Producti...","Gangster, Tragedy, Crime, Drama","Craig Zobel, Kevin Bray, Helen Shaver, Jennife...","Bill Finger, Bob Kane, Lauren LeFranc, Lauren ...",NaN
1,https://www.imdb.com/title/tt7737800/?ref_=sr_t_2,Hẹn Hò Với Sát Nhân,"Anna Kendrick, Daniel Zovatto, Tony Hale, Nico...",1 hour 35 minutes,6.6,"October 4, 2024 (Vietnam)",United States,NaN,NaN,T16,English,"AGC Studios, BondIt Media Capital, Vertigo Ent...","Period Drama, Serial Killer, True Crime, Crime...",Anna Kendrick,Ian McDonald,NaN
2,https://www.imdb.com/title/tt17526714/?ref_=sr...,Thần Dược,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",2 hours 21 minutes,7.6,"November 1, 2024 (Vietnam)",United Kingdom,"$17,500,000 (estimated)","$42,776,354",R,English,"Working Title Films, A Good Story, Blacksmith","Body Horror, Dark Comedy, Monster Horror, Psyc...",Coralie Fargeat,Coralie Fargeat,NaN
3,https://www.imdb.com/title/tt16366836/?ref_=sr...,Venom: Kèo Cuối,"Tom Hardy, Chiwetel Ejiofor, Juno Temple, Rhys...",1 hour 50 minutes,6.2,"October 25, 2024 (Vietnam)",United States,"$120,000,000 (estimated)","$187,339,701",T13,English,"Columbia Pictures, Arad Productions, Hutch Par...","Alien Invasion, Kaiju, Road Trip, Superhero, A...",Kelly Marcel,"Kelly Marcel, Tom Hardy, Kelly Marcel",NaN
4,https://www.imdb.com/title/tt21906238/?ref_=sr...,Rivals,"Alex Hassell, Victoria Smurfit, Bella Maclean,...",55 minutes,8.0,"October 18, 2024 (United Kingdom)",United States,NaN,NaN,TV-MA,English,"Happy Prince, Walt Disney Studios",Drama,"Elliot Hegarty, Alexandra Brodski, Dee Koppang...","Dominic Treadwell-Collins, Laura Wade, Dare Ai...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,https://www.imdb.com/title/tt21064584/?ref_=sr...,The Iron Claw,"Zac Efron, Jeremy Allen White, Harris Dickinso...",2 hours 12 minutes,NaN,"December 22, 2023 (United States)",United Kingdom,"$15,000,000 (estimated)","$45,204,449",R,English,"A24, Access Entertainment, BBC Film","Docudrama, Tragedy, Biography, Drama, Sport",Sean Durkin,Sean Durkin,NaN
1994,https://www.imdb.com/title/tt0119237/?ref_=sr_...,Gummo,"Nick Sutton, Jacob Sewell, Lara Tosh, Jacob Re...",1 hour 29 minutes,NaN,"November 24, 1997 (Australia)",United States,"$1,300,000 (estimated)","$116,799",R,English,"Fine Line Features, Independent Pictures (II)","Dark Comedy, Comedy, Drama",Harmony Korine,Harmony Korine,NaN
1995,https://www.imdb.com/title/tt16300962/?ref_=sr...,Cuando acecha la maldad,"Ezequiel Rodríguez, Demián Salomón, Silvina Sa...",1 hour 39 minutes,NaN,"October 27, 2023 (United States)",Argentina,NaN,NaN,Not Rated,Spanish,"Machaco Films, Aramos Cine, Shudder","Supernatural Horror, Horror",Demián Rugna,Demián Rugna,NaN
1996,https://www.imdb.com/title/tt0110357/?ref_=sr_...,Vua Sư Tử,"Matthew Broderick, Jeremy Irons, James Earl Jo...",1 hour 28 minutes,NaN,"June 24, 1994 (United States)",United States,"$45,000,000 (estimated)","$981,708,285",G,English,"Walt Disney Pictures, Walt Disney Animation St...","Animal Adventure, Coming-of-Age, Hand-Drawn An...","Roger Allers, Rob Minkoff","Irene Mecchi, Jonathan Roberts, Linda Woolvert...",NaN
